In [1]:
import numpy as np
from deepsudoku import REPO_PATH
import os
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import tqdm
import tensorflow as tf
from deepsudoku.supervised_learning import SudukoWrapper, load_supervised_data

os.chdir(REPO_PATH)

2023-09-06 18:11:11.814407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 18:11:12.418501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = load_supervised_data.get_data("test", "realistic_easy_data")
test_data_sets = []

stepsize = len(data)//10

for i in range(10):
    
    test_data_sets.append(load_supervised_data.get_tf_dataset(data[i*stepsize:(i+1)*stepsize]).batch(64).prefetch(tf.data.AUTOTUNE))

2023-09-06 18:11:21.818166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-06 18:11:21.840880: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-06 18:11:21.841088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
names = os.listdir("supervised_experiments/logs")

names

['same_fc_mlp_1693909990.6347742',
 'Random_Data_bigger_big_conv_head_1693915710.1695652',
 'big_conv_head_1693994852.9515347',
 'same_conv_conv_head_better_loss_1693856195.751896',
 'same_conv_with_fc_head_1693912399.9989173',
 'same_conv_conv_head_1693849476.7056108',
 'same_transformer_1693902682.8953056',
 'Group_Data_bigger_big_conv_head_1693983284.6414773']

In [5]:

comp = [n for n in names if "same" in n or n[0]=="b" and "trans" not in n]
print(comp)
to_ckpt = lambda x: os.path.join("supervised_experiments/ckpt/",x,"final_model")


def get_preds(name):
    
    back_bone = tf.keras.models.load_model(to_ckpt(name))
    model = SudukoWrapper(back_bone)

    all_preds = []

    for ds in test_data_sets:

        preds = model.predict(ds, verbose = 0)
        all_preds.append(preds)
    all_preds = np.concatenate(all_preds)
    all_preds = all_preds.reshape(-1, 81)

    return np.array(["".join(ar.astype(str)) for ar in (all_preds)])


for name in tqdm.tqdm(comp):
    
    preds = get_preds(name)
    
    np.save(os.path.join("stats/preds",f"{name}.npy"), preds)
    

['same_fc_mlp_1693909990.6347742', 'big_conv_head_1693994852.9515347', 'same_conv_conv_head_better_loss_1693856195.751896', 'same_conv_with_fc_head_1693912399.9989173', 'same_conv_conv_head_1693849476.7056108', 'same_transformer_1693902682.8953056']


  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [01:15<06:18, 75.74s/it]

 33%|███▎      | 2/6 [03:51<08:10, 122.53s/it]

 50%|█████     | 3/6 [05:14<05:14, 104.90s/it]

 67%|██████▋   | 4/6 [06:34<03:10, 95.06s/it] 

 83%|████████▎ | 5/6 [07:52<01:34, 94.51s/it]


OSError: No file or directory found at supervised_experiments/ckpt/same_transformer_1693902682.8953056/final_model